# 1.Import packages

In [7]:
# Importing all required packages at the start of the notebook
import IPython

from qiime2 import Visualization

import qiime2 as q2
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# 2.Import the data

In [3]:
# Location of the projects data
!mkdir -p "Project_data"
data_dir = "Project_data/Import_and_Denoizing"

In [4]:
%%bash -s $data_dir
mkdir -p "$1"

wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/uV06vmm96ZzB5eM/download/fungut_forward_reads.qza
wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/CA76kKFC9FApqpR/download/fungut_metadata.tsv

chmod -R +rxw "$1"

--2025-10-22 11:09:33--  https://polybox.ethz.ch/index.php/s/uV06vmm96ZzB5eM/download/fungut_forward_reads.qza
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712595535 (680M) [application/octet-stream]
Saving to: ‘Project_data/Import_and_Denoizing/fungut_forward_reads.qza’

     0K ........ ........ ........ ........  4%  356M 2s
 32768K ........ ........ ........ ........  9%  376M 2s
 65536K ........ ........ ........ ........ 14%  375M 2s
 98304K ........ ........ ........ ........ 18%  266M 2s
131072K ........ ........ ........ ........ 23%  571M 1s
163840K ........ ........ ........ ........ 28%  224M 1s
196608K ........ ........ ........ ........ 32%  427M 1s
229376K ........ ........ ........ ........ 37%  487M 1s
262144K ........ ........ ........ ........ 42%  422M 1s
294912K ........ ........ ........ ........ 47%  307M 1s
327

# 3.Feature table construction

In [ ]:
# Visual summary of the data
! qiime demux summarize \
    --i-data $data_dir/fungut_forward_reads.qza \
    --o-visualization $data_dir/fungut_forward_reads_demux_seqs.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
Visualization.load(f"{data_dir}/fungut_forward_reads_demux_seqs.qzv")

Reasoning why lenght was choosen

# Denoising - Amplicon Sequence Variants

In [ ]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
    --p-trunc-len 135 \
    --p-n-threads 3 \
    --p-max-ee 4 \
    --p-min-fold-parent-over-abundance 4 \
    --o-table $data_dir/dada2_table.qza \
    --o-representative-sequences $data_dir/dada2_rep_set.qza \
    --o-denoising-stats $data_dir/dada2_stats.qza

In [ ]:
! qiime metadata tabulate \
  --m-input-file $data_dir/dada2_stats.qza \
  --o-visualization $data_dir/dada2_stats.qzv

! qiime feature-table tabulate-seqs \
  --i-data $data_dir/dada2_rep_set.qza \
  --o-visualization $data_dir/dada2_rep_set.qzv

! qiime feature-table summarize \
  --i-table $data_dir/dada2_table.qza \
  --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
  --o-visualization $data_dir/dada2_table.qzv

In [ ]:
Visualization.load(f"{data_dir}/dada2_stats.qzv")

In [ ]:
Visualization.load(f"{data_dir}/dada2_rep_set.qzv")

In [ ]:
Visualization.load(f"{data_dir}/dada2_table.qzv")